In [1]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import math
import tifffile
from library import helper
from library import plots
percent_of_side = 0.05

In [2]:
_, blue = cv2.imreadmulti('data/Blue-34a-dbl_0002.tiff', [], cv2.IMREAD_UNCHANGED)
_, red = cv2.imreadmulti('data/Red-34a-dbl_0002.tiff', [], cv2.IMREAD_UNCHANGED)
_, composite = cv2.imreadmulti('data/Composite-34a-dbl_0002.tiff', [], cv2.IMREAD_GRAYSCALE)

In [3]:
blue_composite = [(blue[i], composite[i]) for i in range(len(composite))]
red_composite = [(red[i], composite[i]) for i in range(len(composite))]

#Blue
images = []
for idx, (bl, cmp) in enumerate(blue_composite):
    if idx < len(blue_composite)*0.05 or idx > len(blue_composite)*0.95:
        point = helper.get_intensity_pictures(bl, cmp, outer_bool=True)
    else:
        point = helper.get_intensity_pictures(bl, cmp)
    images.append(point)

TypeError: get_intensity_pictures() got multiple values for argument 'outer_bool'

In [4]:
#Intensity images
images = []
# for idx, cmp in enumerate(composite):
for idx, (blue,cmp) in enumerate(blue_composite):
    if idx < len(composite)*percent_of_side or idx > len(composite)*(1-percent_of_side):
        point = helper.get_intensity_pictures(cmp, outer_bool=True)
    else:
        point = helper.get_intensity_pictures(cmp)
    images.append(point)
#Blue
points_blue = []
for idx, (bl, cmp) in enumerate(blue_composite):
    if idx < len(blue_composite)*percent_of_side or idx > len(blue_composite)*(1-percent_of_side):
        point = helper.get_intensity_points(bl, cmp, outer_bool=True)
    else:
        point = helper.get_intensity_points(bl, cmp)
    points_blue.append(point)

#Red
points_red = []
for idx, (rd, cmp) in enumerate(red_composite):
    if idx < len(red_composite)*percent_of_side or idx > len(red_composite)*(1-percent_of_side):
        point = helper.get_intensity_points(rd, cmp, outer_bool=True)
    else:
        point = helper.get_intensity_points(rd, cmp)
    points_red.append(point)


In [5]:
FRAMES = [inner for inner, outer, comp in images] + [outer for inner, outer, comp in images] +  [comp for inner, outer, comp in images]
OUT_NAME = "blue_2.tiff" # Name to save to
with tifffile.TiffWriter(OUT_NAME) as tiff:
    for img in FRAMES:
        tiff.save(img)

/var/folders/9m/qcg58ft12w72z4h1j7s196w00000gn/T/ipykernel_11248/2769783586.py:5: DeprecationWarning: <tifffile.TiffWriter.save> is deprecated. Use TiffWriter.write
  tiff.save(img)


In [6]:
inner_points_blue = [j for i in points_blue for j in i[0]]
inner_points_red = [j for i in points_red for j in i[0]]
outer_points_blue = [j for i in points_blue for j in i[1]]
outer_points_red = [j for i in points_red for j in i[1]]

In [7]:
#Intrinsic and extrinsic noise for inner
#intrinsic = (1/n)*sum([0.5*(blue - red)**2 for blue, red in zip(blue_intensities, red_intensities)])/(mean(blue_intensities) * mean(red_intensities))
#extrinsic = (1/n)*sum([blue*red - mean(blue_intesity)*mean(red_intensity) for blue, red in zip(blue_intensities, red_intensities)])/(mean(blue_intensities) * mean(red_intensities))
#total = (1/n)*sum([0.5*(blue**2 - red**2) - mean(blue_intesity)*mean(red_intensity) for blue, red in zip(blue_intensities, red_intensities)])/(mean(blue_intensities) * mean(red_intensities))
inner_points_blue_mean = np.mean(inner_points_blue)
inner_points_red_mean = np.mean(inner_points_red)
inner_intrinsic = (1/len(inner_points_blue))*sum([0.5*(blue - red)**2 for blue, red in zip(inner_points_blue, inner_points_red)])/(inner_points_blue_mean * inner_points_red_mean)

outer_points_blue_mean = np.mean(outer_points_blue)
outer_points_red_mean = np.mean(outer_points_red)
outer_intrinsic = (1/len(outer_points_blue))*sum([0.5*(blue - red)**2 for blue, red in zip(outer_points_blue, outer_points_red)])/(outer_points_blue_mean * outer_points_red_mean)

/var/folders/9m/qcg58ft12w72z4h1j7s196w00000gn/T/ipykernel_11248/3326364238.py:7: RuntimeWarning: overflow encountered in ushort_scalars
  inner_intrinsic = (1/len(inner_points_blue))*sum([0.5*(blue - red)**2 for blue, red in zip(inner_points_blue, inner_points_red)])/(inner_points_blue_mean * inner_points_red_mean)
/var/folders/9m/qcg58ft12w72z4h1j7s196w00000gn/T/ipykernel_11248/3326364238.py:11: RuntimeWarning: overflow encountered in ushort_scalars
  outer_intrinsic = (1/len(outer_points_blue))*sum([0.5*(blue - red)**2 for blue, red in zip(outer_points_blue, outer_points_red)])/(outer_points_blue_mean * outer_points_red_mean)


In [8]:
inner_extrinsic = (1/len(inner_points_blue))*(sum([blue*red for blue, red in zip(inner_points_blue, inner_points_red)]) - inner_points_red_mean*inner_points_blue_mean) /(inner_points_blue_mean * inner_points_red_mean)
outer_extrinsic = (1/len(outer_points_blue))*(sum([blue*red for blue, red in zip(outer_points_blue, outer_points_red)]) - outer_points_red_mean*outer_points_blue_mean) /(outer_points_blue_mean * outer_points_red_mean)

/var/folders/9m/qcg58ft12w72z4h1j7s196w00000gn/T/ipykernel_11248/3606372120.py:1: RuntimeWarning: overflow encountered in ushort_scalars
  inner_extrinsic = (1/len(inner_points_blue))*(sum([blue*red for blue, red in zip(inner_points_blue, inner_points_red)]) - inner_points_red_mean*inner_points_blue_mean) /(inner_points_blue_mean * inner_points_red_mean)
/var/folders/9m/qcg58ft12w72z4h1j7s196w00000gn/T/ipykernel_11248/3606372120.py:2: RuntimeWarning: overflow encountered in ushort_scalars
  outer_extrinsic = (1/len(outer_points_blue))*(sum([blue*red for blue, red in zip(outer_points_blue, outer_points_red)]) - outer_points_red_mean*outer_points_blue_mean) /(outer_points_blue_mean * outer_points_red_mean)


In [12]:
inner_total = (1/len(inner_points_blue))*(sum([0.5*(blue**2 + red**2) for blue, red in zip(inner_points_blue, inner_points_red)])-  inner_points_red_mean*inner_points_blue_mean)/(inner_points_red_mean*inner_points_blue_mean)
outer_total = (1/len(outer_points_blue))*(sum([0.5*(blue**2 + red**2) for blue, red in zip(outer_points_blue, outer_points_red)])-  outer_points_red_mean*outer_points_blue_mean)/(outer_points_red_mean*outer_points_blue_mean)

In [13]:
inner_intrinsic,inner_extrinsic,inner_total

(953.4647475454173, 0.043239770881738575, 1.490227410844598)

In [14]:
outer_intrinsic,outer_extrinsic,outer_total

(834.8982817516057, 0.03076054307595648, 1.5054462938347886)